#SCC - ICMC - USP
#**MBA EM INTELIGENCIA ARTIFICIAL E BIGDATA**
## **CURSO 2 - CD, AM E DM**
## **PROFA. ROSELI AP. FRANCELIN ROMERO**
---
## **Exemplo - TRANSFORMAÇÕES EM DADOS HETEROGÊNEOS**
**OBJETIVO**:
## COMPLEMENTAR O PIPELINE DE TRANSFORMAÇÃO NOS DADOS

A ideia deste notebook é completar o tópico de transformações em dados heterogêneos, incluindo as funções 'StandardScaler' (normalização Z-score) e 'OneHotEncoder' que transforma dados categóricos em numéricos e usando Pipelines.  Pipelines devem ser usados em qualquer código na área de Ciência de Dados e Aprendizado de Máquina, pois, é uma boa prática de programação.


In [ ]:
# Retomando o mesmo exemplo da Aula Prática_Transformação_Numerico_Numerico
import pandas as pd
import numpy as np
# Dados de exemplo
dados = pd.DataFrame({
    'idade': [15, 23, 19, 30, 44, np.NaN],
    'sexo': ['homem', 'mulher', 'homem', 'homem', np.NaN,'homem'],
    'altura': [1.6, 1.7, np.NaN, 1.8, 1.75, 1.65],
    'classe':  [-1, 1, -1, 1, 1, -1]
    })
dados

,idade,sexo,altura,classe
0,15.0,homem,1.60,-1
1,23.0,mulher,1.70,1
2,19.0,homem,NaN,-1
3,30.0,homem,1.80,1
4,44.0,NaN,1.75,1
5,NaN,homem,1.65,-1


In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   idade   5 non-null      float64
 1   sexo    5 non-null      object 
 2   altura  5 non-null      float64
 3   classe  6 non-null      int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 320.0+ bytes


A partir do comando `info()` nota-se que existem 2 variáveis de entrada numéricas (idade e altura) e 1 variável de entrada categórica (sexo). A variável classe é a variável alvo.

### Transformando os dados

Considere que queremos realizar as seguintes operações:


*   **Nos dados numéricos:** substituir os dados faltantes das variáveis pela média dos valores presentes. Depois padronizar o intervalo dessas variáveis
*   **Nos dados categóricos:** Substituir os valores faltantes pelo valor mais frequente. Depois transformar essas categorias em um atributo numérico para usar no nosso modelo de Aprendizado de Máquina.

Para fazer essas operações em colunas específicas, podemos utilizar as ferramentas 'sklearn.pipeline.Pipeline' e 'sklearn.compose.ColumnTransformer'.

A ferramenta 'sklearn.pipeline.Pipeline' cria uma sequencia de transformações nos dados, enquanto que a `sklearn.compose.ColumnTransformer` realiza uma transformação em dados de uma coluna.

Para substituir valores faltantes utilizaremos a classe 'sklearn.impute.SimpleImputer', para padronizar os dados 'sklearn.preprocessingStandardScaler' e para converter atributos categóricos em valores numéricos 'sklearn.preprocessing.OneHotEncoder'.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# Criamos um vetor com o nome das classes desejadas
features_numericos = ['idade', 'altura']
features_categoricos = ['sexo']

# Criando os pipelines
pipeline_numerico = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
     ('scaler', StandardScaler())])

pipeline_categorico = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())])

# Criando a transformação do conjunto de dados:
transformacao = ColumnTransformer(
    transformers=[
        ('transformacao numerica', pipeline_numerico, features_numericos),
        ('transformacao categorica', pipeline_categorico, features_categoricos),
    ])

# Aplicando a transformação no dataset:
dados_transformados = transformacao.fit_transform(dados)


In [ ]:
dados_transformados

array([[-1.20446356e+00, -1.54919334e+00,  1.00000000e+00,
         0.00000000e+00],
       [-3.44132445e-01, -3.43990023e-15,  0.00000000e+00,
         1.00000000e+00],
       [-7.74298001e-01, -3.43990023e-15,  1.00000000e+00,
         0.00000000e+00],
       [ 4.08657278e-01,  1.54919334e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 1.91423672e+00,  7.74596669e-01,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -7.74596669e-01,  1.00000000e+00,
         0.00000000e+00]])

Note que a ordem das variáveis (features) mudou em relação ao nosso conjunto inicial. A ordem do conjunto transformado é dada pela ordem de processamento das features. No nosso caso, a ordem das features ficou:

(features_numericos, features_categoricos) $\rightarrow$ (idade, altura, sexo)

 Note também que a transformação `OneHotEncoder` vai transformar os features categóricos em um vetor unitário, que possui $n$ dimensões, onde $n$ é o número de valores diferentes da categoria (no nosso exemplo, 2 - homem e mulher)

 Observe que o conjunto não possui mais a variável "classe". Isso porque não foi efetuada nenhuma transformação nessa variável.

 Normalmente para problemas de Machine Learning é comum separar as variáveis de entrada ("sexo", "idade" e "altura") da variável alvo ("classe"). Mas se for necessário, podemos facilmente concatenar a variável classe e criar uma estrutura `DataFrame`:

In [ ]:
dados_transformados_com_classe = np.c_[dados_transformados, dados['classe']]
dataframe_processado = pd.DataFrame(dados_transformados_com_classe)
dataframe_processado

,0,1,2,3,4
0,-1.204464,-1.549193e+00,1.0,0.0,-1.0
1,-0.344132,-3.439900e-15,0.0,1.0,1.0
2,-0.774298,-3.439900e-15,1.0,0.0,-1.0
3,0.408657,1.549193e+00,1.0,0.0,1.0
4,1.914237,7.745967e-01,1.0,0.0,1.0
5,0.000000,-7.745967e-01,1.0,0.0,-1.0




---


**Opcional:** para adicionar nomes às variáveis do dataframe:

In [ ]:
nomes = []
# Variáveis numéricas não tem alteração de tamanho, logo:
nomes = nomes + features_numericos

# Para os dados categóricos devemos acessar o transformador:
transformacao_categorica = transformacao.transformers_[1]
# Depois o pipeline
pipeline_categorico = transformacao_categorica[1]
# E finalmente o onehot
transf_onehot = pipeline_categorico.named_steps['onehot']
# Para acessar o nome das variáveis usamos o método get_feature_names()
nomes = nomes + (transf_onehot.get_feature_names_out().tolist())

nomes.append("classe")

dataframe_processado = pd.DataFrame(data = dados_transformados_com_classe, columns=nomes)
dataframe_processado.round(2)

,idade,altura,x0_homem,x0_mulher,classe
0,-1.20,-1.55,1.0,0.0,-1.0
1,-0.34,-0.00,0.0,1.0,1.0
2,-0.77,-0.00,1.0,0.0,-1.0
3,0.41,1.55,1.0,0.0,1.0
4,1.91,0.77,1.0,0.0,1.0
5,0.00,-0.77,1.0,0.0,-1.0
